<a href="https://colab.research.google.com/github/abdullahmujahidali/Vet-LangGraph/blob/main/VetAI_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langgraph langchain_openai openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00


In [ ]:
import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
import json

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

In [20]:
class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str
    medical_records: Dict[str, Any]
    owner_history: Dict[str, Any]
    diagnoses: Dict[str, Any]
    results: Dict[str, Any]


def create_data_processing_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a veterinary data processing agent. Extract structured information from "
                  "medical records, lab reports, and other documents. Organize content based on document "
                  "type and relevant sections including patient information, history, clinical notes, "
                  "medications, and treatments."),
        ("human", "Process the following input: {input}")
    ])

    def process_data(state: VetState):
        messages = prompt.format_messages(input=state["current_input"])
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["DataProcessor"] = response.content
        return new_state

    return process_data

def create_history_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a veterinary history analysis agent. Extract and summarize the pet's "
                  "medical history including past conditions, treatments, medications, and owner "
                  "constraints. Track medication history with precise dosing details."),
        ("human", "Based on this processed data: {processed_data}\n\nAnalyze the patient history.")
    ])

    def analyze_history(state: VetState):
        processed_data = state["results"]["DataProcessor"]
        messages = prompt.format_messages(processed_data=processed_data)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["HistoryAnalyzer"] = response.content
        return new_state

    return analyze_history

def create_clinical_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a veterinary clinical record analyst. Review and analyze medical records "
                  "to create comprehensive summaries, track patient details, organize lab results, "
                  "and document owner preferences."),
        ("human", "Given this history analysis: {history_analysis}\n\nProvide a clinical analysis.")
    ])

    def analyze_clinical_records(state: VetState):
        history_analysis = state["results"]["HistoryAnalyzer"]
        messages = prompt.format_messages(history_analysis=history_analysis)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["ClinicalAnalyzer"] = response.content
        return new_state

    return analyze_clinical_records

def create_diagnostic_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a veterinary diagnostic intelligence agent. Analyze and categorize medical "
                  "problems with confidence levels, correlate findings between owner reports and medical "
                  "records, and identify patterns in the case."),
        ("human", "Based on this clinical analysis: {clinical_analysis}\n\nProvide diagnostic insights.")
    ])

    def diagnose(state: VetState):
        clinical_analysis = state["results"]["ClinicalAnalyzer"]
        messages = prompt.format_messages(clinical_analysis=clinical_analysis)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["Diagnostics"] = response.content
        return new_state

    return diagnose


data_processor = create_data_processing_agent()
history_analyzer = create_history_analysis_agent()
clinical_analyzer = create_clinical_analysis_agent()
diagnostic_agent = create_diagnostic_agent()

In [21]:
workflow = StateGraph(VetState)

workflow.add_node("process_data", data_processor)
workflow.add_node("analyze_history", history_analyzer)
workflow.add_node("analyze_clinical", clinical_analyzer)
workflow.add_node("diagnose", diagnostic_agent)

workflow.set_entry_point("process_data")

workflow.add_edge("process_data", "analyze_history")
workflow.add_edge("analyze_history", "analyze_clinical")
workflow.add_edge("analyze_clinical", "diagnose")
workflow.add_edge("diagnose", END)

app = workflow.compile()

In [22]:
# Test the veterinary workflow
initial_state = {
    "messages": [],
    "current_input": """
    Patient Information:
    Name: Max
    Species: Canine
    Breed: Golden Retriever
    Age: 8 years

    History:
    Presenting with chronic limping on right front leg
    Previous treatment with Carprofen 75mg BID for 2 weeks
    X-rays taken 3 months ago showed mild arthritis

    Current Medications:
    Glucosamine/Chondroitin supplement daily

    Owner Notes:
    Difficulty with long walks
    Still eager to play but tires quickly
    Financial constraints noted for advanced imaging
    """,
    "medical_records": {},
    "owner_history": {},
    "diagnoses": {},
    "results": {}
}

result = app.invoke(initial_state)

print("\nData Processing Results:")
print(result["results"]["DataProcessor"])
print("\nHistory Analysis:")
print(result["results"]["HistoryAnalyzer"])
print("\nClinical Analysis:")
print(result["results"]["ClinicalAnalyzer"])
print("\nDiagnostic Assessment:")
print(result["results"]["Diagnostics"])


Data Processing Results:
### Patient Information:
- **Name:** Max
- **Species:** Canine
- **Breed:** Golden Retriever
- **Age:** 8 years

### History:
- Presenting with chronic limping on right front leg
- Previous treatment with Carprofen 75mg BID for 2 weeks
- X-rays taken 3 months ago showed mild arthritis

### Current Medications:
- Glucosamine/Chondroitin supplement daily

### Owner Notes:
- Difficulty with long walks
- Still eager to play but tires quickly
- Financial constraints noted for advanced imaging

History Analysis:
### Summary of Patient History:
- **Name:** Max
- **Species:** Canine
- **Breed:** Golden Retriever
- **Age:** 8 years
- **Presenting Complaint:** Chronic limping on right front leg
- **Previous Treatment:** Carprofen 75mg BID for 2 weeks
- **Past Condition:** Mild arthritis observed on X-rays taken 3 months ago
- **Current Medications:** Glucosamine/Chondroitin supplement daily
- **Owner Constraints:** Financial limitations for advanced imaging
- **Owner No